
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src=https://raw.githubusercontent.com/aestaire/ml_workshop/refs/heads/main/files/images/hands-on.png>
</div>

## Agent Bricks: Creando un Knowledge Assistant

Agent Bricks en Databricks te permite construir asistentes inteligentes que acceden y analizan información relevante de tus datos empresariales. Con Agent Bricks, puedes crear Knowledge Assistants capaces de responder preguntas, extraer insights y proporcionar análisis contextualizados de manera automática.

En este notebook aprenderás a:
* Configurar un agente especializado utilizando Agent Bricks en Databricks;
* Integrar documentos para enriquecer el conocimiento del asistente;
* Personalizar el comportamiento del agente para responder consultas específicas y generar análisis.

Esta funcionalidad te ayuda a transformar la información en conocimiento accionable, facilitando la toma de decisiones y mejorando la eficiencia en el acceso a datos clave.

En el siguiente ejemplo, _crea_ un esquema con tu nombre para no tener problema de sobreescritura de datos:

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS `workshop_databricks`.`demo_agentbricks_nombre`; -- sustituye nombre por el tuyo

CREATE VOLUME IF NOT EXISTS `workshop_databricks`.`demo_agentbricks_nombre`.`archivos`; -- sustituye nombre por el tuyo

In [0]:
catalog = "workshop_databricks"
schema = "demo_agentbricks_nombre" #sustituye nombre por el tuyo
volume = "archivos"

file_name = "economia_mundial.pdf"
table_name = "economia_mundial_pdf"
download_url = "https://raw.githubusercontent.com/aestaire/ml_workshop/refs/heads/main/files/data/economia_mundial.pdf"
path_volume = f"/Volumes/{catalog}/{schema}/{volume}"
path_table = f"{catalog}.{schema}"

print(path_table)   # Show the complete path
print(path_volume)  # Show the complete path

# Copy the CSV file from the URL to the volume
dbutils.fs.cp(download_url, f"{path_volume}/{file_name}")

In [0]:
from pyspark.sql.functions import expr

# Full path for standard Python libraries (like PyPDF)
full_file_path = f"/Volumes/{catalog}/{schema}/{volume}/{file_name}"

# Path for Spark APIs
path_volume = f"/Volumes/{catalog}/{schema}/{volume}"

# Path for table creation
path_table = f"{catalog}.{schema}"

df = spark.read.format("binaryFile").load(full_file_path).withColumn(
    "parsed",
    expr("ai_parse_document(content)"))

display(df)

In [0]:
# Extract all columns from dataframe
from pyspark.sql.functions import col, parse_json

df_text = df.withColumn(
   "parsed_json",
   parse_json(col("parsed").cast("string"))) \
 .selectExpr(
   "path",
   "parsed_json:document:elements")

display(df_text)

In [0]:
#Explode into multiple rows with content from each page
from pyspark.sql.functions import explode, from_json
from pyspark.sql.types import ArrayType, StringType

# Define the expected array type for your data
array_schema = ArrayType(StringType())

# Convert 'elements' (VARIANT) to array by parsing as JSON string
df_text2 = df_text.withColumn("elements_array", from_json(col("elements").cast("string"), array_schema))

# Explode the new array column
df_text3 = df_text2.select("path", explode(col("elements_array")).alias("element"))
display(df_text3)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Add id to the dataframe
df_text4 = df_text3.withColumn("id", monotonically_increasing_id())
display(df_text4)

In [0]:
# Write dataframe to Delta Table
df_text4.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workshop_databricks.demo_agentbricks.economia_mundial_pdf")

## Crear Vector Search Index

Un Vector Search Index es una herramienta que permite buscar información usando "vectores", que son representaciones numéricas de textos, imágenes o datos. Así, puedes encontrar rápidamente documentos o respuestas similares a lo que preguntas, incluso si usas palabras diferentes.

In [0]:
catalog = "workshop_databricks"
schema = "demo_agentbricks"
table_name = "economia_mundial_pdf"

workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
table_link = f"{workspace_url}/explore/data/{catalog}/{schema}/{table_name}"

displayHTML(f"Access your table here: {table_link}")

<img src="./files/images/vsi01.png" width="350">

<img src="./files/images/vsi02.png" width="500">

## Crear Agent Bricks

<img src="./files/images/agents.png" width="300">

<img src="./files/images/KA.png" width="500">

<img src="./files/images/agentbricks.png" width="1000">

**Description**

Agente especializado en el informe 'Actualización de Perspectivas de la Economía Mundial'. Proporciona análisis y datos clave sobre la resiliencia de la economía y la incertidumbre persistente.

**Describe the content**

Informe sobre los últimos datos de crecimiento, proyecciones de inflación o riesgos económicos.

**Clica en "Create Agent"**

### Playground

Playground es un espacio interactivo donde puedes probar y explorar el Knowledge Assistant que creamos. Aquí puedes hacer preguntas sobre el informe de economía mundial y recibir respuestas automáticas basadas en los datos y análisis disponibles. Solo tienes que escribir tu consulta y el asistente te ayudará con información relevante de manera sencilla y rápida.

**Clica en "Open in Playground"**

<img src="./files/images/KA2.png" width="1000">

#### Preguntas

* Cuáles son las proyecciones económicas para Brasil en 2026?
* Y para México?
* Cuáles son las políticas para restablecer la confianza y garantizar la sostenibilidad?

## ¡Felicidades! ¡Has completado tu primer laboratorio de ML, AI functions y Agent bricks!

¡Ahora ya sabes cómo utilizar Unity Catalog, entrenar y registrar modelos con MLflow, usar las AI functions y crear un agente utilizando Agent Bricks!